In [268]:
import pandas as pd
import numpy as np
import os

RANDOM_STATE = 404
number_of_samples = 10

In [269]:
df = pd.read_csv('data/cardio_train.csv', delimiter=';')
df.drop(columns=['id'], inplace=True)
df.head(5)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [270]:
X = df.drop(columns=['cardio'])
y = df['cardio']

In [271]:
# Create an empty dictionary to store the results after each method
results_dict = {}

## Data manipulation

#### Standardization

In [272]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Select columns to be scaled
numeric_columns = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'gender', 'cholesterol']
categorical_columns = ['gluc', 'smoke', 'alco', 'active']

# Fit and transform your data (only for numeric columns)
scaler = StandardScaler()
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

# Apply one-hot encoding to categorical columns
label_encoder = LabelEncoder()
for col in categorical_columns:
    X[col] = label_encoder.fit_transform(X[col])

X.head(5)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,-0.436062,1.364055,0.443452,-0.847873,-0.122182,-0.088238,-0.539322,0,0,0,1
1,0.307686,-0.733108,-1.018168,0.749831,0.072610,-0.035180,2.400793,0,0,0,1
2,-0.247997,-0.733108,0.078047,-0.708942,0.007679,-0.141297,2.400793,0,0,0,0
3,-0.748152,1.364055,0.565254,0.541435,0.137541,0.017879,-0.539322,0,0,0,1
4,-0.808543,-0.733108,-1.018168,-1.264666,-0.187113,-0.194356,-0.539322,0,0,0,0


In [273]:
X.describe()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
count,7.000000e+04,7.000000e+04,7.000000e+04,7.000000e+04,7.000000e+04,7.000000e+04,7.000000e+04,70000.000000,70000.000000,70000.000000,70000.000000
mean,5.272227e-16,-1.644399e-16,1.450116e-15,-2.905105e-16,7.623108e-17,1.745905e-17,1.381498e-16,0.226457,0.088129,0.053771,0.803729
std,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,0.572270,0.283484,0.225568,0.397179
min,-3.514407e+00,-7.331083e-01,-1.332014e+01,-4.460075e+00,-1.810381e+00,-8.841161e-01,-5.393221e-01,0.000000,0.000000,0.000000,0.000000
25%,-7.315341e-01,-7.331083e-01,-6.527630e-01,-6.394770e-01,-5.725127e-02,-8.823850e-02,-5.393221e-01,0.000000,0.000000,0.000000,1.000000
50%,9.489744e-02,-7.331083e-01,7.804703e-02,-1.532192e-01,-5.725127e-02,-8.823850e-02,-5.393221e-01,0.000000,0.000000,0.000000,1.000000
75%,7.531244e-01,1.364055e+00,6.870554e-01,5.414349e-01,7.261016e-02,-3.517999e-02,9.307354e-01,0.000000,0.000000,0.000000,1.000000
max,1.720199e+00,1.364055e+00,1.043119e+01,8.738353e+00,1.031826e+02,5.785165e+01,2.400793e+00,2.000000,1.000000,1.000000,1.000000


#### Splitting

In [274]:
from sklearn.model_selection import train_test_split

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)
X_train.shape, X_test.shape

((52500, 11), (17500, 11))

## Preparing 10 subsets with removed features

In [275]:
# Function that randomly removes features and replace their values with NaN
def remove_features(num_features_to_remove=None, feature_indices_to_remove=None):
    """
    Randomly removes features from a subset of data and replaces their values with NaN.
    
    Parameters:
        num_features_to_remove (int): Number of features to remove randomly.
        feature_indices_to_remove (array-like): Indices of features to remove.
        
    Returns:
        pandas.DataFrame: Subset of data with removed features and NaN values.
    """
    # Sample a subset of data
    subset = X_train.sample(frac=0.001, random_state=RANDOM_STATE)
    
    # Determine features to remove based on number or indices provided
    if feature_indices_to_remove is None:
        if num_features_to_remove is None:
            num_features_to_remove = np.random.randint(1, min(5, len(X_train.columns) - 1))
        features_to_remove = np.random.choice(subset.columns[:-1], num_features_to_remove, replace=False)
    else:
        features_to_remove = subset.columns[feature_indices_to_remove]
    
    # Replace values of selected features with NaN
    features_to_remove = np.random.choice(subset.columns[:-1], num_features_to_remove, replace=False)
    subset = subset.astype(object)
    subset.loc[:, features_to_remove] = np.nan
    
    return subset

list_of_subsets = []
subset_without_changes = X_train.sample(frac=0.001, random_state=RANDOM_STATE)

# Generate subsets with varying numbers of removed features
for _ in range(2):
    list_of_subsets.append(remove_features(1))

for _ in range(2):
    list_of_subsets.append(remove_features(2))

for _ in range(2):
    list_of_subsets.append(remove_features(3))

for _ in range(2):
    list_of_subsets.append(remove_features(4))

for _ in range(2):
    list_of_subsets.append(remove_features(np.random.randint(5, 7)))

# Print information about subsets and their missing columns
print(f'Subsets with {list_of_subsets[0].shape[0]} datapoints and their columns with missing values:')
for subset_index, subset in enumerate(list_of_subsets):
    nan_columns = subset.columns[subset.isnull().all()]
    print(f"Subset {subset_index+1}: {', '.join(nan_columns)}")

Subsets with 52 datapoints and their columns with missing values:
Subset 1: age
Subset 2: cholesterol
Subset 3: height, alco
Subset 4: age, smoke
Subset 5: height, ap_hi, alco
Subset 6: age, height, smoke
Subset 7: height, ap_hi, ap_lo, alco
Subset 8: gender, height, weight, smoke
Subset 9: age, ap_hi, ap_lo, gluc, smoke
Subset 10: age, gender, height, weight, gluc, smoke


## Shared functions

#### Imputation

In [276]:
from sklearn.impute import SimpleImputer
from ConditionalGMM.condGMM import CondGMM
import json

def imputing_missing_data(subsets, method='simple', model=None):
    for subset in subsets:
        if method == 'simple':
            # Simple Imputer
            generated_data = simple_impute(subset)
            continue
        
        # Initialize to keep track of actual row index, because indices were shuffled
        row_in_subset_index = 0
        
        for row_index, row in subset.iterrows():
            # Get indices and values of unknown and known features
            missing_features_indices = [row.index.get_loc(col) for col in row.index if pd.isna(row[col])]
            
            # If all features are known, continue
            if len(missing_features_indices) == 0:
                continue
            
            generated_data = None
            
            if method == 'multivariate' or method == 'cgmm':
                # Multivariate Imputer or Conditional GMM
                generated_data = cgmm_impute(model, missing_features_indices, row_in_subset_index)
                continue
            elif method == 'vae':
                # Variational AutoEncoder
                generated_data = vae_impute(model, missing_features_indices, row)
            
            # Update unknown features with sampled data
            for feature_index in range(len(missing_features_indices)):
                if subset.columns[missing_features_indices[feature_index]] in categorical_columns:
                    # Approximate categorical values to the nearest whole number
                    generated_data[:, feature_index] = np.round(generated_data[:, feature_index])
                subset.iloc[row_in_subset_index, missing_features_indices[feature_index]] = json.dumps([generated_data[sample_index][feature_index] for sample_index in range(generated_data.shape[0])])
            
            row_in_subset_index += 1

def simple_impute(current_subset):
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')

    for col in current_subset.columns:
        if pd.isna(current_subset[col]).any():
            imp.fit(X_train[[col]])
            current_subset[col] = imp.transform(current_subset[[col]])

def cgmm_impute(gmm, missing_features_indices, row_in_subset_index):
    # Find indices of known features
    known_features_indexes = list(set(range(subset.shape[1])) - set(missing_features_indices))
    
    # Extract values of known features for the given row
    known_features_values = subset.iloc[row_in_subset_index, known_features_indexes]
    
    # Initialize CondGMM
    cGMM = CondGMM(gmm.weights_, gmm.means_, gmm.covariances_, known_features_indexes)
    
    # Generate samples using Conditional GMM
    generated_data = cGMM.rvs(known_features_values, size=number_of_samples, random_state=RANDOM_STATE)
    
    return generated_data

def vae_impute(model, missing_features_indices, current_row):
    generated_data = np.empty((number_of_samples, len(missing_features_indices)))
        
    # Repeat the prediction process for the specified number of samples
    for _ in range(number_of_samples):
        imputed_values_row = []
        # Impute missing values for each feature index
        for feature_index in missing_features_indices:
            # Impute missing value using the VAE for the current feature and row
            imputed_value = model.predict(current_row.values.reshape(1, -1).astype(np.float32), verbose=0)[0, feature_index]
            imputed_values_row.append(imputed_value)
        generated_data[_] = imputed_values_row
        
    return generated_data

#### Scoring

In [277]:
import json

def get_scoring(subsets, method, should_print=False):
    """
    Calculate Mean Squared Error (MSE) scores for features in subsets of data.
    
    Parameters:
        subsets (list): List of subsets of data.
        method (str): Method used for imputation ('simple', 'multivariate', 'cgmm', or 'vae').
        should_print (bool): Whether to print MSE scores or not.
        
    Returns:
        dict: Dictionary containing MSE scores for each feature in the subsets.
    """    
    method = method.lower()
    if method == 'multivariate' or method == 'cgmm':
        # Convert serialized arrays in each subset to lists
        for subset in subsets:
            for col in subset.columns:
                subset[col] = subset[col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # Iterate through subsets
    for subset_index, subset in enumerate(subsets):
        feature_score = {}  # Initialize dictionary to store MSE values
        
        # Determine unknown features indexes dynamically for each subset
        if method == 'simple':
            missing_features_indices = [col_index for col_index, col in enumerate(list_of_subsets[subset_index].columns) if list_of_subsets[subset_index][col].isnull().all()]
        else:
            missing_features_indices = [col_index for col_index, col in enumerate(subset.columns) if subset[col].apply(lambda x: isinstance(x, list)).any()]

        if not missing_features_indices:
            continue  # Skip if there are no missing values
        
        # Iterate through rows in the subset DataFrame
        for index, row in subset.iterrows():
            original_values = X.iloc[index, missing_features_indices].values
            
            # Compute MSE for each feature separately
            for feature_index in range(len(missing_features_indices)):
                if method == 'simple':
                    generated_samples = [row.iloc[missing_features_indices].values[feature_index]]
                else:    
                    generated_samples_raw = row.iloc[missing_features_indices].values[feature_index]
                    generated_samples = [sample for sample in generated_samples_raw if not pd.isna(sample)]
                
                # Grab the original value of the feature
                original_value = original_values[feature_index]
                
                for sample in generated_samples:
                    squared_error = (original_value - sample)**2
                    
                    if missing_features_indices[feature_index] not in feature_score:
                        feature_score[missing_features_indices[feature_index]] = []
                        
                    feature_score[missing_features_indices[feature_index]].append(squared_error)
        
        for feature_index, score_values in feature_score.items():
            mse = np.mean(score_values)
            # Not sure if it should be X, X_train, or subset_without_changes
            variance = np.var(subset_without_changes.iloc[:, feature_index])
            nmse = mse / variance
            feature_score[feature_index] = nmse
        
        # Print MSE scores if required
        if should_print:
            print(f"MSE for Subset {subset_index + 1}:")
            for feature_index, score_values in feature_score.items():
                print(f"Feature {subset_without_changes.columns[feature_index]}: MSE = {score_values}")
    
    # Return dictionary containing MSE scores
    return feature_score


#### Classification

In [278]:
from joblib import load
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings('ignore', message="X does not have valid feature names")

# Load the trained classifier model
classifier = load('classifiers\cardio_classifier.h5')

def get_accuracy(subsets, method, should_print=False):
    """
    Calculate accuracy scores for subsets of data using a trained classifier.
    
    Parameters:
        subsets (list): List of subsets of data.
        method (str): Method used for imputation ('simple', 'multivariate', or 'cgmm').
        should_print (bool): Whether to print accuracy scores or not.
        
    Returns:
        list: List of accuracy scores for each subset.
    """
    method = method.lower()
    
    if method == 'multivariate' or method == 'cgmm':
        # Convert serialized arrays in each subset to lists
        for subset in subsets:
            for col in subset.columns:
                subset[col] = subset[col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    classification_results = []  # Initialize list to store classification results
    accuracy_per_subset = []  # Initialize list to store accuracy scores

    # Iterate through subsets
    for subset_index, subset in enumerate(subsets):
        subset_results = []  # Initialize list to store results for the current subset
        
        # Iterate through rows in the subset DataFrame
        for row_index, row in subset.iterrows():
            row_results = []  # Initialize list to store results for the current row
            
            # Process each row based on the method used
            if method != 'simple':
                serialized_arrays = []
                non_serialized_values = []
                
                # Split row values into serialized arrays and non-serialized values
                for col, value in row.items():
                    if isinstance(value, list):
                        serialized_arrays.append((col, value))
                    else:
                        non_serialized_values.append((col, value))
                
                # Generate combined rows by combining serialized arrays with non-serialized values
                for i in range(number_of_samples):
                    combined_row = non_serialized_values.copy()
                    
                    for col, serialized_array in serialized_arrays:
                        if i < len(serialized_array):
                            combined_row.append((col, serialized_array[i]))
                    
                    combined_row_array = [value for _, value in combined_row]
                    
                    try:
                        result_array = classifier.predict([combined_row_array], verbose=0)
                        row_results.append(result_array)
                    except Exception as e:
                        print(f"Error processing row {row_index}: {e}")
            else:
                # For simple method, predict directly from row values
                result_array = classifier.predict([row.values.tolist()], verbose=0)
                row_results.append(result_array)

            subset_results.append(row_results)  # Append results for the current row to the subset results
        
        classification_results.append(subset_results)  # Append subset results to the classification results

    # Calculate accuracy scores for each subset
    for subset_index, subset_results in enumerate(classification_results):
        true_labels = y.loc[subsets[subset_index].index]  # Get true labels for the current subset
        
        subset_predicted_labels = []  # Initialize list to store predicted labels for the subset
        
        # Determine predicted labels for each row in the subset
        for row_results in subset_results:
            predicted_label = 1 if row_results[0] > 0.5 else 0  # Assuming threshold of 0.5
            subset_predicted_labels.append(predicted_label)
        
        subset_accuracy = accuracy_score(true_labels, subset_predicted_labels)  # Calculate accuracy score for the subset
        
        accuracy_per_subset.append(subset_accuracy)  # Append accuracy score to the list

    # Print accuracy scores if required
    if (should_print):
        for subset_index, subset_accuracy in enumerate(accuracy_per_subset):
            print("Subset", subset_index+1, "accuracy:", subset_accuracy)
        
    return accuracy_per_subset  # Return list of accuracy scores for each subset


## SimpleImputer with mean strategy

#### Preparation

In [ ]:
import copy

imputer_subsets = copy.deepcopy(list_of_subsets)

#### Imputation

In [ ]:
imputing_missing_data(imputer_subsets, 'simple')

imputer_subsets[0].head(5)

#### Scoring

In [ ]:
simple_imputer_score = get_scoring(imputer_subsets, 'simple')

#### Classification

In [ ]:
simple_imputer_accuracy = get_accuracy(imputer_subsets, 'simple', True)

In [ ]:
results_dict['simple_imputer'] = {'score': simple_imputer_score, 'accuracy': simple_imputer_accuracy}

## Multivariate normal distribution

#### Preparation

In [ ]:
multivariate_subsets = copy.deepcopy(list_of_subsets)

In [ ]:
from sklearn.mixture import GaussianMixture

# Create Gaussian Mixture Model with a single component
gmm = GaussianMixture(n_components=1, random_state=RANDOM_STATE)
gmm.fit(X_train)

#### Imputation

In [ ]:
imputing_missing_data(multivariate_subsets, 'multivariate', gmm)

multivariate_subsets[0].head(5)

#### Scoring

In [ ]:
multivariate_score = get_scoring(multivariate_subsets, 'multivariate')

#### Classification

In [ ]:
multivariate_accuracy = get_accuracy(multivariate_subsets, 'multivariate', True)

In [ ]:
results_dict['multivariate'] = {'score': multivariate_score, 'accuracy': multivariate_accuracy}

## Conditional GMM

#### Preparation

In [ ]:
cgmm_subsets = copy.deepcopy(list_of_subsets)

In [ ]:
# Using BIC to get the optimal number of components for GMM

import matplotlib.pyplot as plt

def compute_bic(data, n_components_range):
    """
    Computes the Bayesian Information Criterion (BIC) for Gaussian Mixture Models with different numbers of components.
    
    Parameters:
        X (array-like): Input data.
        n_components_range (range): Range of number of components to evaluate.
        
    Returns:
        list: BIC values for each number of components.
    """
    # List to store BIC values
    bic = []
    
    for n_components in n_components_range:
        # Create Gaussian Mixture Model with specified number of components
        gmm = GaussianMixture(n_components=n_components, random_state=RANDOM_STATE)
        gmm.fit(data)  # Fit the model to the data
        bic.append(gmm.bic(data))  # Calculate BIC and add to list
        
    return bic  # Return list of BIC values

n_components_range = range(1, 51)  # Range of number of components to evaluate
bic_values = compute_bic(X_train, n_components_range)  # Compute BIC values
optimal_n_components = n_components_range[np.argmin(bic_values)]  # Determine optimal number of components

# Plotting BIC values
plt.plot(n_components_range, bic_values, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('BIC Value')
plt.title('BIC for Gaussian Mixture Models')
plt.grid(True)
# plt.savefig('images/BIC_without_missingness.png')
plt.show()

In [ ]:
# Create Gaussian Mixture Model with optimal number of components
gmm = GaussianMixture(n_components=optimal_n_components, random_state=RANDOM_STATE)
gmm.fit(X_train)

#### Imputation

In [ ]:
imputing_missing_data(cgmm_subsets, 'cgmm', gmm)

cgmm_subsets[0].head(5)

#### Scoring

In [ ]:
cgmm_score = get_scoring(cgmm_subsets, 'cgmm')

#### Classification

In [ ]:
cgmm_accuracy = get_accuracy(cgmm_subsets, 'cgmm', True)

In [ ]:
results_dict['cgmm'] = {'score': cgmm_score, 'accuracy': cgmm_accuracy}

## Variational Autoencoder

#### Preparation

In [279]:
vae_subsets = copy.deepcopy(list_of_subsets)

In [264]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Lambda
# from tensorflow.keras.models import Model

# # Define function to compute negative log likelihood
# def compute_nll(model, X_test):
#     reconstructions = model.predict(X_test)
#     mse = np.mean(np.square(X_test - reconstructions), axis=1)
#     nll = 0.5 * np.log(2 * np.pi * mse)
#     return np.mean(nll)

# # Define range of latent space dimensionalities to try
# latent_dim_range = range(2, 5)

# # Train VAE models with different latent space dimensionalities and model capacities
# vae_models = {}
# for latent_dim in latent_dim_range:
#     for num_layers in [1, 2, 3, 4]:
#         for num_neurons in [32, 64, 128]:
#             # Define the encoder
#             input_dim = X_train.shape[1]
#             inputs = Input(shape=(input_dim,))
#             encoded = inputs
#             for i in range(num_layers):
#                 encoded = Dense(num_neurons / (2**(i-1)), activation='relu')(encoded)
#             z_mean = Dense(latent_dim)(encoded)
#             z_log_var = Dense(latent_dim)(encoded)

#             # Reparameterization trick
#             def sampling(args):
#                 z_mean, z_log_var = args
#                 epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
#                 return z_mean + tf.exp(0.5 * z_log_var) * epsilon

#             z = Lambda(sampling)([z_mean, z_log_var])

#             # Define the decoder
#             decoded = z
#             for i in range(num_layers):
#                 decoded = Dense(num_neurons / (2**(i-1)), activation='relu')(decoded)
#             outputs = Dense(input_dim)(decoded)

#             # Create the VAE model
#             vae = Model(inputs, outputs)

#             # Compile the model
#             vae.compile(optimizer='adam', loss='mse')  # Use MSE as the reconstruction loss

#             # Train the model
#             history = vae.fit(X_train, X_train, epochs=10, batch_size=32, verbose=0)

#             # Evaluate performance on test set
#             nll_test = compute_nll(vae, X_test)
#             print(f"Latent Dim: {latent_dim}, Num Layers: {num_layers}, Num Neurons: {num_neurons}, Test NLL: {nll_test}")

#             # Store model and its performance
#             vae_models[(latent_dim, num_layers, num_neurons)] = {'model': vae, 'nll_test': nll_test}

# # Select model with lowest mean NLL on test set
# best_config = min(vae_models, key=lambda x: vae_models[x]['nll_test'])
# best_model = vae_models[best_config]['model']

# print(f"Best Model Configuration: Latent Dim = {best_config[0]}, Num Layers = {best_config[1]}, Num Neurons = {best_config[2]}, Test NLL = {vae_models[best_config]['nll_test']}")

In [265]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model

latent_dim = 4

input_dim = X_train.shape[1]
inputs = Input(shape=(input_dim,))
encoded = inputs
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)
z_mean = Dense(latent_dim)(encoded)
z_log_var = Dense(latent_dim)(encoded)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Define the decoder
decoded = z
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
outputs = Dense(input_dim)(decoded)

# Create the VAE model
vae = Model(inputs, outputs)

# Compile the model
vae.compile(optimizer='adam', loss='mse')  # Use MSE as the reconstruction loss

# Train the model
history = vae.fit(X_train, X_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
1641/1641 [==============================] - 4s 2ms/step - loss: 0.1860
Epoch 2/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.1323
Epoch 3/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0931
Epoch 4/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0529
Epoch 5/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0660
Epoch 6/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0463
Epoch 7/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0525
Epoch 8/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0359
Epoch 9/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0503
Epoch 10/10
1641/1641 [==============================] - 3s 2ms/step - loss: 0.0295


#### Imputation

In [280]:
imputing_missing_data(vae_subsets, 'vae', vae)

vae_subsets[0].head(5)

#### Scoring

In [ ]:
vae_score = get_scoring(vae_subsets, 'vae')

#### Classification

In [ ]:
vae_accuracy = get_accuracy(vae_subsets, 'vae', True)

In [ ]:
results_dict['vae'] = {'score': vae_score, 'accuracy': vae_accuracy}

## Comparison of results

In [ ]:
from tabulate import tabulate

# Convert accuracy values to percentages
for key, value in results_dict.items():
    results_dict[key]["accuracy"] = [round(acc * 100, 2) for acc in value["accuracy"]]

# Create a table
table = [[""] + list(results_dict.keys())]
for i in range(10):
    table.append([i+1] + [results_dict[key]["accuracy"][i] for key in results_dict.keys()])

# Print the table
print(tabulate(table, headers="firstrow", tablefmt="grid"))